# Abel & Josette's AI Term Project
This is our workspace for collaborative filtering and content based music generation.
For spotify API, reference this page for [setup](https://medium.com/@benfenison/using-spotify-web-api-w-python-6b31a328b26e) and this page for [documentation](https://github.com/plamere/spotipy).

In [15]:
import sys
import spotipy
import spotipy.util as util

scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username,scope,client_id='33ebd71536d94caebcf54970eaf4191a',
                                  client_secret='87c219c0663d414388f96a6b7da7ee11', redirect_uri='http://localhost/')

In [16]:
sp_us = spotipy.Spotify(auth=token)

In [19]:
results = sp_us.search(q='weezer', limit=20)
for i, t in enumerate(results['tracks']['items']):
    print (' ', i, t['name'])

  0 Say It Ain't So
  1 Island In The Sun
  2 Buddy Holly
  3 Beverly Hills
  4 Africa
  5 Undone - The Sweater Song
  6 The End of the Game
  7 Hash Pipe
  8 My Name Is Jonas
  9 Feels Like Summer
  10 Pork And Beans
  11 Take On Me
  12 El Scorcho
  13 Everybody Wants To Rule The World
  14 Perfect Situation
  15 Only In Dreams
  16 Mr. Blue Sky
  17 The World Has Turned And Left Me Here
  18 In The Garage
  19 Can't Knock The Hustle


In [32]:

''' shows recommendations for the given artist
'''

from spotipy.oauth2 import SpotifyClientCredentials
#client_credentials_manager = SpotifyClientCredentials()
#sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp = spotipy.Spotify(auth=token)
sp.trace=False

def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None

def show_recommendations_for_artist(artist):
    albums = []
    results = sp.recommendations(seed_artists = [artist['id']])
    #print(results)
    for track in results['tracks']:
        #print("what")
        print(track['name'], '-', track['artists'][0]['name'])

if __name__ == '__main__':
    musicer = input("Please enter an Artist: ")
    if len(musicer) <= 0:
        print(('Usage: {0} artist name'.format(sys.argv[0])))
    else:
        #name = ' '.join(musicer)
        name = musicer
        #print(name)
        artist = get_artist(name)
        #print(artist)
        if artist:
            show_recommendations_for_artist(artist)
        else:
            print("Can't find that artist", name)

Please enter an Artist: Rihanna
FourFiveSeconds - Rihanna
Broken & Beautiful (from the movie UGLYDOLLS) - Kelly Clarkson
oui - Jeremih
Unholy - Miley Cyrus
Don't Check On Me (feat. Justin Bieber & Ink) - Chris Brown
Feels - Kehlani
American Oxygen - Rihanna
CAN'T STOP THE FEELING! (Original Song from DreamWorks Animation's "TROLLS") - Justin Timberlake
Sorry Not Sorry - Demi Lovato
Wild Ones (feat. Sia) - Flo Rida
Fallin' - Alicia Keys
Diamonds - Rihanna
Sally Walker - Iggy Azalea
Whatcha Say - Jason Derulo
Don't Cha - The Pussycat Dolls
Love On The Brain - Rihanna
Bed (feat. Ariana Grande) - Nicki Minaj
Apologize - Timbaland
I Kissed A Girl - Katy Perry
Power is Power (feat. The Weeknd & Travis Scott) - SZA
